# Estimating employee wages in Toronto

This information will be used as a feature to analyze why housing prices are high in Toronto.<br>

Source:<br>
•	Employee wages by occupation, monthly from 1997 to 2020: https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1410030601 <br>


In [14]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math



In [4]:
df_wage = pd.read_csv(r'C:\Users\aamini\Personal\Job applications\Validere\Data Scientiist_Feb2022\Validere technical assessment\Data\All Datasets\Stat Canada\Employee wages by occupation, monthly, unadjusted for seasonality\14100306.csv')

## Let's see what we have here:

In [6]:
df_wage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30394980 entries, 0 to 30394979
Data columns (total 19 columns):
 #   Column                                      Dtype  
---  ------                                      -----  
 0   REF_DATE                                    object 
 1   GEO                                         object 
 2   DGUID                                       object 
 3   Wages                                       object 
 4   Type of work                                object 
 5   National Occupational Classification (NOC)  object 
 6   Sex                                         object 
 7   Age group                                   object 
 8   UOM                                         object 
 9   UOM_ID                                      int64  
 10  SCALAR_FACTOR                               object 
 11  SCALAR_ID                                   int64  
 12  VECTOR                                      object 
 13  COORDINATE               

### Very large file!

In [5]:
df_wage.head()

,REF_DATE,GEO,DGUID,Wages,Type of work,National Occupational Classification (NOC),Sex,Age group,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1997-01,Canada,2016A000011124,"Total employees, all wages",Both full- and part-time employees,"Total employees, all occupations",Both sexes,15 years and over,Persons,249,thousands,3,v103449834,1.1.1.1.1.1,10921.0,NaN,NaN,NaN,1
1,1997-01,Canada,2016A000011124,"Total employees, all wages",Both full- and part-time employees,"Total employees, all occupations",Both sexes,15 to 24 years,Persons,249,thousands,3,v103449835,1.1.1.1.1.2,1721.8,NaN,NaN,NaN,1
2,1997-01,Canada,2016A000011124,"Total employees, all wages",Both full- and part-time employees,"Total employees, all occupations",Both sexes,25 to 54 years,Persons,249,thousands,3,v103449836,1.1.1.1.1.3,8333.1,NaN,NaN,NaN,1
3,1997-01,Canada,2016A000011124,"Total employees, all wages",Both full- and part-time employees,"Total employees, all occupations",Both sexes,55 years and over,Persons,249,thousands,3,v103449837,1.1.1.1.1.4,866.2,NaN,NaN,NaN,1
4,1997-01,Canada,2016A000011124,"Total employees, all wages",Both full- and part-time employees,"Total employees, all occupations",Males,15 years and over,Persons,249,thousands,3,v103449838,1.1.1.1.2.1,5645.4,NaN,NaN,NaN,1


In [13]:
df_wage.GEO.unique()

array(['Canada', 'Newfoundland and Labrador', 'Prince Edward Island',
       'Nova Scotia', 'New Brunswick', 'Quebec', 'Ontario', 'Manitoba',
       'Saskatchewan', 'Alberta', 'British Columbia'], dtype=object)

## Parmeters of interest and assumptions:

* GEO: The information is available on provincial scale. **Assumption: Employee wages in Toronto correlate with Ontario.** <br>
* Wages: Only "Total employees, all wages" is considered.<br>
* Type of work : Only "Both full- and part-time employees" is considered.<br>
* Sex: "Both sexes" are considered.<br>
* Age group: No division based on age os done; only "15 years and over" is considered.<br>
* National Occupational Classification (NOC): all occupations are considered together.<br>
* SCALAR_FACTOR: Thousand Canadian dollars.<br>
* VALUE: All employees wages value for the province in any given month.<br>



## Let's filter the database for parameters of interest in the window of 2011 - 2020

In [16]:
df = df_wage.loc[(df_wage['GEO'] == 'Ontario') & (df_wage['Wages'] == 'Total employees, all wages') & 
                      (df_wage['Age group'] == '15 years and over') & (df_wage['Sex'] == 'Both sexes') &
                     (df_wage['National Occupational Classification (NOC)'] == 'Total employees, all occupations')&
                (df_wage['REF_DATE'] >= '2011-01') & (df_wage['REF_DATE'] <= '2021-12') & 
                 (df_wage['Type of work'] == 'Both full- and part-time employees')]

In [6]:
df.head()

,REF_DATE,GEO,DGUID,Wages,Type of work,National Occupational Classification (NOC),Sex,Age group,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
17019720,2011-01,Ontario,2016A000235,"Total employees, all wages",Both full- and part-time employees,"Total employees, all occupations",Both sexes,15 years and over,Persons,249,thousands,3,v103504914,7.1.1.1.1.1,5517.1,NaN,NaN,NaN,1
17120700,2011-02,Ontario,2016A000235,"Total employees, all wages",Both full- and part-time employees,"Total employees, all occupations",Both sexes,15 years and over,Persons,249,thousands,3,v103504914,7.1.1.1.1.1,5540.8,NaN,NaN,NaN,1
17221680,2011-03,Ontario,2016A000235,"Total employees, all wages",Both full- and part-time employees,"Total employees, all occupations",Both sexes,15 years and over,Persons,249,thousands,3,v103504914,7.1.1.1.1.1,5543.4,NaN,NaN,NaN,1
17322660,2011-04,Ontario,2016A000235,"Total employees, all wages",Both full- and part-time employees,"Total employees, all occupations",Both sexes,15 years and over,Persons,249,thousands,3,v103504914,7.1.1.1.1.1,5596.2,NaN,NaN,NaN,1
17423640,2011-05,Ontario,2016A000235,"Total employees, all wages",Both full- and part-time employees,"Total employees, all occupations",Both sexes,15 years and over,Persons,249,thousands,3,v103504914,7.1.1.1.1.1,5704.3,NaN,NaN,NaN,1


In [7]:
df.to_csv('OntarioEmploymentWages_2011-2021.csv')